In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from skopt import gp_minimize
from skopt.space import Integer, Real
from sklearn.preprocessing import MinMaxScaler
import shap

import os
os.environ['PYTHONHASHSEED'] = '42'

# Output directory
output_dir = "Excel_XGBoost_Results"
os.makedirs(output_dir, exist_ok=True)

# Load and normalize features
FEATURE_COLUMNS = [
    "C", "H", "O", "N", "F", "S", "System_Size", "a", "b", "c",
    "density", "PLD", "LCD", "N2_SA",
    "Probe_Accessible", "Probe_Occupiable", "Rosenbluth_Weight"
]
data = pd.read_csv("PIM_ExpFeatures.csv")
features = data[FEATURE_COLUMNS].values
labels = pd.read_csv("PIM_Qst_Labels.csv")["Qst_CO2_298K"].values

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# Define search space
#space = [
#    Integer(50, 300, name="n_estimators"),
#    Integer(2, 20, name="max_depth"),
#    Real(0.01, 0.3, name="learning_rate"),
#    Real(0.5, 1.0, name="subsample"),
#    Real(0.5, 1.0, name="colsample_bytree")
#]

# Objective function
#def objective(params):
#    n_estimators, max_depth, learning_rate, subsample, colsample_bytree = params
#    r2_scores = []

#    kf = KFold(n_splits=5, shuffle=True, random_state=42)
#    for train_idx, test_idx in kf.split(features):
#        X_train, X_test = features[train_idx], features[test_idx]
#        y_train, y_test = labels[train_idx], labels[test_idx]

#        model = XGBRegressor(
#            n_estimators=n_estimators,
#            max_depth=max_depth,
#            learning_rate=learning_rate,
#            subsample=subsample,
#            colsample_bytree=colsample_bytree,
#            objective="reg:squarederror",
#            n_jobs=1,  # <- Enforce single-threaded determinism
#            verbosity=0,
#            random_state=42,
#            tree_method="exact",  # <- Optional: exact split for full determinism
#            enable_categorical=False  # <- In case categorical creep in
#        )

#        model.fit(X_train, y_train)
#        preds = model.predict(X_test)
#        r2_scores.append(r2_score(y_test, preds))

#    return -np.mean(r2_scores)

# Run Bayesian optimization
#result = gp_minimize(
#    objective,
#    space,
#    n_calls=500,
#    n_initial_points=50,
#    random_state=42,
#    verbose=True
#)

# Best hyperparameters
#best_params = result.x
#param_names = ["n_estimators", "max_depth", "learning_rate", "subsample", "colsample_bytree"]
#best_params_dict = dict(zip(param_names, best_params))
#pd.DataFrame([best_params_dict]).to_csv(os.path.join(output_dir, "XGBoost_best_hyperparameters.csv"), index=False)

# Final model evaluation
train_r2s, test_r2s = [], []
all_train_actuals, all_train_preds = [], []
all_test_actuals, all_test_preds = [], []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, test_idx in kf.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    model = XGBRegressor(
        n_estimators=109,
        max_depth=18,
        learning_rate=0.179388327,
        subsample=0.5,
        colsample_bytree=1,
        n_jobs=1,
        verbosity=0,
        tree_method="exact",
        random_state=42,
        enable_categorical=False
    )
    model.fit(X_train, y_train)
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    all_train_actuals.extend(y_train)
    all_train_preds.extend(train_preds)
    all_test_actuals.extend(y_test)
    all_test_preds.extend(test_preds)

    train_r2s.append(r2_score(y_train, train_preds))
    test_r2s.append(r2_score(y_test, test_preds))

# Save CV scores
#cv_df = pd.DataFrame({
#    "Fold": range(1, 6),
#    "Training R^2": train_r2s,
#    "Testing R^2": test_r2s
#})
#cv_df.to_csv(os.path.join(output_dir, "XGBoost_cv_results.csv"), index=False)

# Final parity plot with both training and testing results
plt.figure(figsize=(6, 6))
plt.rcParams.update({'font.family': 'Times New Roman'})

# Scatter points
plt.scatter(all_train_actuals, all_train_preds, color='blue', alpha=0.7, edgecolor='k', label=f"Training ($R^2$ = {np.mean(train_r2s):.3f})")
plt.scatter(all_test_actuals, all_test_preds, color='red', alpha=0.7, edgecolor='k', label=f"Testing ($R^2$ = {np.mean(test_r2s):.3f})")

# Diagonal reference line
min_val = min(min(all_train_actuals), min(all_test_actuals))
max_val = max(max(all_train_actuals), max(all_test_actuals))
plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)

# Labels and title
plt.xlabel("Actual Qst", fontsize=14)
plt.ylabel("Predicted Qst", fontsize=14)
plt.title("XGBoost Prediction Results", fontsize=14)

# Styling
plt.xticks(fontsize=12, color='black')
plt.yticks(fontsize=12, color='black')
plt.grid(True, linestyle='--', linewidth=0.5)
plt.legend(loc='upper left', fontsize=12, frameon=False)
plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()

# Save as PDF
plt.savefig(os.path.join(output_dir, "XGBoost_combined_parity_plot.pdf"), format='pdf')
plt.close()

In [3]:
# Retrain best CatBoost model on full dataset
final_model = XGBRegressor(
    n_estimators=109,
    max_depth=18,
    learning_rate=0.179388327,
    subsample=0.5,
    colsample_bytree=1,
    objective="reg:squarederror",
    n_jobs=1,
    verbosity=0,
    tree_method="exact",
    random_state=42, 
    enable_categorical=False
)
final_model.fit(features, labels)

# SHAP analysis
explainer = shap.Explainer(final_model)
shap_values = explainer(features)

# Save SHAP values (one row per sample, one column per feature)
shap_df = pd.DataFrame(shap_values.values, columns=FEATURE_COLUMNS)
shap_df["Sample_Index"] = np.arange(len(shap_df))
shap_df.to_csv(os.path.join(output_dir, "XGBoost_SHAP_values.csv"), index=False)

# Compute and save feature ranking (mean absolute SHAP)
mean_abs_shap = np.abs(shap_df[FEATURE_COLUMNS]).mean().sort_values(ascending=False)
mean_abs_shap_df = mean_abs_shap.reset_index()
mean_abs_shap_df.columns = ["Feature", "Mean_Absolute_SHAP"]
mean_abs_shap_df.to_csv(os.path.join(output_dir, "XGBoost_SHAP_feature_ranking.csv"), index=False)

print("✅ SHAP analysis completed and saved.")

✅ SHAP analysis completed and saved.


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

# Set global font to Times New Roman and base font size to 14
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 14

# Load SHAP feature ranking
shap_ranking_path = os.path.join(output_dir, "XGBoost_SHAP_feature_ranking.csv")
shap_ranking = pd.read_csv(shap_ranking_path)

# Plot: Top 10 SHAP features
plt.figure(figsize=(8, 6))
barplot = sns.barplot(
    data=shap_ranking.head(10),
    x="Mean_Absolute_SHAP",
    y="Feature",
    orient="h",
    palette="viridis"
)

# Adjust axis labels and tick label sizes
barplot.set_title("Top 10 Feature Importances by SHAP (XGBoost)", fontsize=14)
barplot.set_xlabel("Mean Absolute SHAP Value", fontsize=14)
barplot.set_ylabel("Feature", fontsize=14)
barplot.tick_params(axis='x', labelsize=12)
barplot.tick_params(axis='y', labelsize=14)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the figure
plot_path = os.path.join(output_dir, "XGBoost_SHAP_feature_importance_plot.png")
plt.savefig(plot_path, dpi=300)
plt.close()

print(f"✅ SHAP feature importance plot saved to: {plot_path}")


/var/folders/3q/n2zkzqzd5q9c607jywjkk4tc0000gn/T/ipykernel_76065/130876771.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(


✅ SHAP feature importance plot saved to: Excel_XGBoost_Results/XGBoost_SHAP_feature_importance_plot.png
